## 1. Import the Data Wrangling dataset and clean the data

In [ ]:
import pandas as pd

In [ ]:
#http://dmip.webs.upv.es/datawrangling/catalog.html
df = pd.read_csv("all_in_one.csv")

In [ ]:
df.head()

,dataset,domain,input_1,input_2,input_3,expected_output
0,addPunctuation-1,dates,290386,NaN,NaN,29-03-86
1,addPunctuation-1,dates,250374,NaN,NaN,25-03-74
2,addPunctuation-1,dates,170615,NaN,NaN,17-06-15
3,addPunctuation-1,dates,170905,NaN,NaN,17-09-05
4,addPunctuation-1,dates,241206,NaN,NaN,24-12-06


In [ ]:
df.describe()

,dataset,domain,input_1,input_2,input_3,expected_output
count,731,731,731,210,36,721
unique,119,7,485,94,15,560
top,convert-3,freetext,17:55,425,UTC,hotmail.com
freq,12,146,12,18,14,14


In [ ]:
df['expected_output'] = df['expected_output'].str.lstrip()

In [ ]:
df["input_2"].fillna("Null", inplace = True)
df["input_3"].fillna("Null", inplace = True)
df["expected_output"].fillna("", inplace = True)

In [ ]:
df

,dataset,domain,input_1,input_2,input_3,expected_output
0,addPunctuation-1,dates,290386,Null,Null,29-03-86
1,addPunctuation-1,dates,250374,Null,Null,25-03-74
2,addPunctuation-1,dates,170615,Null,Null,17-06-15
3,addPunctuation-1,dates,170905,Null,Null,17-09-05
4,addPunctuation-1,dates,241206,Null,Null,24-12-06
...,...,...,...,...,...,...
726,getValue-2,units,41.8g,Null,Null,41.8
727,getValue-2,units,98 ms,Null,Null,98
728,getValue-2,units,4054kA,Null,Null,4054
729,getValue-2,units,92.26 K,Null,Null,92.26


In [ ]:
domains = df['domain'].drop_duplicates().to_list()

## 2. Perform the problem and example sampling and build the different GPT-3 prompts

In [ ]:
import random as rn

final_problems = []

for i in range(0, len(domains)):
  print("Domain: " + domains[i])
  sel_prob = df[['dataset','domain']].query('domain == "' + domains[i] + '"')['dataset'].drop_duplicates().to_list()
  print("Problems: " + str(sel_prob))
  for j in range(0,len(sel_prob)):
    sel_prob[j] = [sel_prob[j],domains[i]]
  final_problems = final_problems + sel_prob


Domain: dates
Problems: ['addPunctuation-1', 'addPunctuation-2', 'changeFormat-1', 'changeFormat-2', 'changeFormat-3', 'changeFormat-4', 'changePunctuation-1', 'changePunctuation-2', 'getDay-1', 'getDay-2', 'getDay-3', 'getDayOrdinal-1', 'getDayOrdinal-2', 'getMonthName-1', 'getMonthName-2', 'getWeekDay-1', 'getWeekDay-2', 'reduceMonthName-1', 'reduceMonthName-2', 'setFormat-1', 'setFormat-2']
Domain: emails
Problems: ['generate-1', 'generate-2', 'generate-3', 'getAfterAt-1', 'getAfterAt-2', 'getAfterAt-3', 'getAfterAt-4', 'getDomain-1', 'getDomain-2', 'someBeforeAt-NA']
Domain: freetext
Problems: ['afterSymbol-1', 'afterSymbol-2', 'betweenSymbols-1', 'betweenSymbols-2', 'brackets-1', 'brackets-2', 'deletePunctuation-1', 'deletePunctuation-2', 'deletePunctuation-3', 'deletePunctuation-4', 'deleteSpaces-1', 'deleteSpaces-2', 'digitToEnd-1', 'digitToEnd-2', 'firstCharacter-1', 'firstCharacter-2', 'getAfterComma-1', 'getAfterComma-2', 'getBetweenCommas-1', 'getBetweenCommas-2', 'getCaps-1

In [ ]:
len(final_problems)

123

In [ ]:
for i in range(0, len(final_problems)):
  if len(df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_1'].to_list()) < 6:
    print(i)

24
41
44


In [ ]:
del final_problems[44]
del final_problems[41]
del final_problems[24]

In [ ]:
for i in range(0, len(final_problems)):
  if len(df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_1'].to_list()) > 6:
    print(i)

47
62
65


In [ ]:
del final_problems[65]
del final_problems[62]
del final_problems[47]

In [ ]:
sum = 0
for i in range(0, len(domains)):
  for j in range(0, len(final_problems)):
    if final_problems[j][1] == domains[i]:
      sum = sum + 1
  print("Domain " + domains[i] + ": " + str(sum))
  sum = 0

Domain dates: 21
Domain emails: 9
Domain freetext: 22
Domain names: 13
Domain phones: 18
Domain times: 24
Domain units: 10


In [ ]:
#task_explanation = ['Set the date input into the format "mm/dd/yyyy"', 'Set the date input into the format "dd/mm/yyyy"','Extract the month from the date input', 'Set the date input into the format "dd-mm-yyyy"','Generate the email address based on the inputs', 'Generate the email address based on the inputs','Extract the information after the first . and before the @ in the email address','Extract the information after the first "." and before the "@" in the email address','Extract the information from the inputs', 'Write the whole input text in capital letters', 'Add the missing brackets to bin the input text', 'Get the content after the comma from the input', 'Add the corresponding title to the provided name', 'Reduce the first name from the input, using just the first letter. Ignore the possible titles', "Extract the professional title from the input. In case there is no professional title, don't return anything", 'Reduce the first name from the input, using just the first letter. Ignore the possible titles', 'Add the country prefix to the phone number', 'Add the country prefix to the phone number', 'Add the country prefix to the phone number','Separate the phone number following the structure "XXX-XXX-XXXX"','Add the seconds to the time input, being the seconds "00"','Get only the numerical part of the time input', 'Add the seconds to the time input in case they are not already included, being the seconds "30"','Delete the seconds from the time input', 'Convert the measure from the initial units to the target units', 'Convert the measure from the initial units to the target units', 'Get the input value without the units', 'Get the input value without the units']

In [ ]:
#Prompt generator - 1 shot

queries = [[]]*len(final_problems)
common_prompt = [""]*len(final_problems)
test_targets = [[]]*len(final_problems)

trainingNum = 1

for i in range(0, len(final_problems)):
  #Extract examples from DataFrame 
  input1 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_1'].to_list()
  input2 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_2'].to_list()
  input3 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_3'].to_list()
  output = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['expected_output'].to_list()

  final_input_1 = []
  final_input_2 = []
  final_input_3 = []
  final_output = []

  #Extract

  for j in range(0, trainingNum):
    final_input_1.append(input1[0])
    del input1[0]

  for j in range(0, trainingNum):
    final_input_2.append(input2[0])
    del input2[0]

  for j in range(0, trainingNum):
    final_input_3.append(input3[0])
    del input3[0]

  for j in range(0, trainingNum):
    final_output.append(output[0])
    del output[0]

  #Common prompt builder

  #If we don't want the task descriptions in the prompt, comment the next line
  #common_prompt[i] = task_explanation[i] + "\n\n"
  for j in range(0, trainingNum):
      if final_input_2[j] != "Null" and final_input_3[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "; " + str(final_input_3[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      elif final_input_2[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      else:
        common_prompt[i] = common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"


  #Queries building

  queries[i] = [""]*len(input1)
  test_targets[i] = [""]*len(input1)

  for j in range(0, len(input1)):
      if input2[j] != "Null" and input3[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "; " + str(input3[j]) + "\nOutput:"
      elif input2[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "\nOutput:"
      else:
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "\nOutput:"

      test_targets[i][j] = output[j]
  

In [ ]:
printed_queries = []
for i in range(0, len(queries)):
  for j in range(0, len(queries[i])):
    printed_queries.append(queries[i][j])

print(printed_queries)

['Input: 290386\nOutput: 29-03-86\n\nInput: 250374\nOutput:', 'Input: 290386\nOutput: 29-03-86\n\nInput: 170615\nOutput:', 'Input: 290386\nOutput: 29-03-86\n\nInput: 170905\nOutput:', 'Input: 290386\nOutput: 29-03-86\n\nInput: 241206\nOutput:', 'Input: 290386\nOutput: 29-03-86\n\nInput: 290607\nOutput:', 'Input: 121099\nOutput: 12-10-99\n\nInput: 90508\nOutput:', 'Input: 121099\nOutput: 12-10-99\n\nInput: 31108\nOutput:', 'Input: 121099\nOutput: 12-10-99\n\nInput: 91011\nOutput:', 'Input: 121099\nOutput: 12-10-99\n\nInput: 290895\nOutput:', 'Input: 121099\nOutput: 12-10-99\n\nInput: 180413\nOutput:', 'Input: 20.08.2010\nOutput: 08/20/2010\n\nInput: 2010-06-27\nOutput:', 'Input: 20.08.2010\nOutput: 08/20/2010\n\nInput: 24/09/1984\nOutput:', 'Input: 20.08.2010\nOutput: 08/20/2010\n\nInput: 13-03-1986\nOutput:', 'Input: 20.08.2010\nOutput: 08/20/2010\n\nInput: 17/06/1989\nOutput:', 'Input: 20.08.2010\nOutput: 08/20/2010\n\nInput: 1990.18.01\nOutput:', 'Input: 09/05/1991\nOutput: 09/05/199

In [ ]:
text_file = open("queries.txt", "w")
for i in range(0, len(printed_queries)):
  text_file.write(repr(printed_queries[i]))
text_file.close()

In [ ]:
#Prompt generator - 2 shot

queries = [[]]*len(final_problems)
common_prompt = [""]*len(final_problems)
test_targets2 = [[]]*len(final_problems)

trainingNum = 2

for i in range(0, len(final_problems)):
  #Extract examples from DataFrame 
  input1 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_1'].to_list()
  input2 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_2'].to_list()
  input3 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_3'].to_list()
  output = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['expected_output'].to_list()

  final_input_1 = []
  final_input_2 = []
  final_input_3 = []
  final_output = []

  #Extract

  for j in range(trainingNum-1,-1,-1):
    final_input_1.append(input1[j])
    del input1[j]

  for j in range(trainingNum-1,-1,-1):
    final_input_2.append(input2[j])
    del input2[j]

  for j in range(trainingNum-1,-1,-1):
    final_input_3.append(input3[j])
    del input3[j]

  for j in range(trainingNum-1,-1,-1):
    final_output.append(output[j])
    del output[j]

  #Common prompt builder

  #If we don't want the task descriptions in the prompt, comment the next line
  #common_prompt[i] = task_explanation[i] + "\n\n"
  for j in range(0, trainingNum):
      if final_input_2[j] != "Null" and final_input_3[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "; " + str(final_input_3[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      elif final_input_2[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      else:
        common_prompt[i] = common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"


  #Queries building

  queries[i] = [""]*len(input1)
  test_targets2[i] = [""]*len(input1)

  for j in range(0, len(input1)):
      if input2[j] != "Null" and input3[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "; " + str(input3[j]) + "\nOutput:"
      elif input2[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "\nOutput:"
      else:
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "\nOutput:"

      test_targets2[i][j] = output[j]


  
  

In [ ]:
printed_queries = []
for i in range(0, len(queries)):
  for j in range(0, len(queries[i])):
    printed_queries.append(queries[i][j])

print(printed_queries)

['Input: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 170615\nOutput:', 'Input: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 170905\nOutput:', 'Input: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 241206\nOutput:', 'Input: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 290607\nOutput:', 'Input: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 31108\nOutput:', 'Input: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 91011\nOutput:', 'Input: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 290895\nOutput:', 'Input: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 180413\nOutput:', 'Input: 2010-06-27\nOutput: 27/06/2010\n\nInput: 20.08.2010\nOutput: 08/20/2010\n\nInput: 24/09/1984\nOutput:', 'Input: 2010-06-27\nOutput: 27/06/2010\n\nInput: 20.08.2010\nOutput: 08/20/2010\n\nInput: 13-03-1986\nOutput:', 'Input: 2010-

In [ ]:
text_file = open("queries2.txt", "w")
for i in range(0, len(printed_queries)):
  text_file.write(repr(printed_queries[i]))
text_file.close()

In [ ]:
#Prompt generator - 3 shot

queries = [[]]*len(final_problems)
common_prompt = [""]*len(final_problems)
test_targets3 = [[]]*len(final_problems)

trainingNum = 3

for i in range(0, len(final_problems)):
  #Extract examples from DataFrame 
  input1 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_1'].to_list()
  input2 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_2'].to_list()
  input3 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_3'].to_list()
  output = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['expected_output'].to_list()

  final_input_1 = []
  final_input_2 = []
  final_input_3 = []
  final_output = []

  #Extract

  for j in range(trainingNum-1,-1,-1):
    final_input_1.append(input1[j])
    del input1[j]

  for j in range(trainingNum-1,-1,-1):
    final_input_2.append(input2[j])
    del input2[j]

  for j in range(trainingNum-1,-1,-1):
    final_input_3.append(input3[j])
    del input3[j]

  for j in range(trainingNum-1,-1,-1):
    final_output.append(output[j])
    del output[j]

  #Common prompt builder

  #If we don't want the task descriptions in the prompt, comment the next line
  #common_prompt[i] = task_explanation[i] + "\n\n"
  for j in range(0, trainingNum):
      if final_input_2[j] != "Null" and final_input_3[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "; " + str(final_input_3[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      elif final_input_2[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      else:
        common_prompt[i] = common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"


  #Queries building

  queries[i] = [""]*len(input1)
  test_targets3[i] = [""]*len(input1)

  for j in range(0, len(input1)):
      if input2[j] != "Null" and input3[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "; " + str(input3[j]) + "\nOutput:"
      elif input2[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "\nOutput:"
      else:
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "\nOutput:"

      test_targets3[i][j] = output[j]


  
  

In [ ]:
printed_queries = []
for i in range(0, len(queries)):
  for j in range(0, len(queries[i])):
    printed_queries.append(queries[i][j])

print(printed_queries)

['Input: 170615\nOutput: 17-06-15\n\nInput: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 170905\nOutput:', 'Input: 170615\nOutput: 17-06-15\n\nInput: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 241206\nOutput:', 'Input: 170615\nOutput: 17-06-15\n\nInput: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 290607\nOutput:', 'Input: 31108\nOutput: 03-11-08\n\nInput: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 91011\nOutput:', 'Input: 31108\nOutput: 03-11-08\n\nInput: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 290895\nOutput:', 'Input: 31108\nOutput: 03-11-08\n\nInput: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 180413\nOutput:', 'Input: 24/09/1984\nOutput: 09/24/1984\n\nInput: 2010-06-27\nOutput: 27/06/2010\n\nInput: 20.08.2010\nOutput: 08/20/2010\n\nInput: 13-03-1986\nOutput:', 'Input: 24/09/1984\nOutput: 09/24/1984\n\nInput: 2010-06-27\nOutput

In [ ]:
text_file = open("queries3.txt", "w")
for i in range(0, len(printed_queries)):
  text_file.write(repr(printed_queries[i]))
text_file.close()

In [ ]:
#Prompt generator - 4 shot

queries = [[]]*len(final_problems)
common_prompt = [""]*len(final_problems)
test_targets4 = [[]]*len(final_problems)

trainingNum = 4

for i in range(0, len(final_problems)):
  #Extract examples from DataFrame 
  input1 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_1'].to_list()
  input2 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_2'].to_list()
  input3 = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['input_3'].to_list()
  output = df.query('dataset == "' + final_problems[i][0] + '" and domain == "' + final_problems[i][1] + '"')['expected_output'].to_list()

  final_input_1 = []
  final_input_2 = []
  final_input_3 = []
  final_output = []

  #Extract

  for j in range(trainingNum-1,-1,-1):
    final_input_1.append(input1[j])
    del input1[j]

  for j in range(trainingNum-1,-1,-1):
    final_input_2.append(input2[j])
    del input2[j]

  for j in range(trainingNum-1,-1,-1):
    final_input_3.append(input3[j])
    del input3[j]

  for j in range(trainingNum-1,-1,-1):
    final_output.append(output[j])
    del output[j]

  #Common prompt builder

  #If we don't want the task descriptions in the prompt, comment the next line
  #common_prompt[i] = task_explanation[i] + "\n\n"
  for j in range(0, trainingNum):
      if final_input_2[j] != "Null" and final_input_3[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "; " + str(final_input_3[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      elif final_input_2[j] != "Null":
        common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "; " + str(final_input_2[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"
      else:
        common_prompt[i] = common_prompt[i] = common_prompt[i] + "Input: " + str(final_input_1[j]) + "\nOutput: " + str(final_output[j]) + "\n\n"


  #Queries building

  queries[i] = [""]*len(input1)
  test_targets4[i] = [""]*len(input1)

  for j in range(0, len(input1)):
      if input2[j] != "Null" and input3[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "; " + str(input3[j]) + "\nOutput:"
      elif input2[j] != "Null":
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "; " + str(input2[j]) + "\nOutput:"
      else:
        queries[i][j] = common_prompt[i] + "Input: " + str(input1[j]) + "\nOutput:"

      test_targets4[i][j] = output[j]


  
  

In [ ]:
printed_queries = []
for i in range(0, len(queries)):
  for j in range(0, len(queries[i])):
    printed_queries.append(queries[i][j])

print(printed_queries)

['Input: 170905\nOutput: 17-09-05\n\nInput: 170615\nOutput: 17-06-15\n\nInput: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 241206\nOutput:', 'Input: 170905\nOutput: 17-09-05\n\nInput: 170615\nOutput: 17-06-15\n\nInput: 250374\nOutput: 25-03-74\n\nInput: 290386\nOutput: 29-03-86\n\nInput: 290607\nOutput:', 'Input: 91011\nOutput: 09-10-11\n\nInput: 31108\nOutput: 03-11-08\n\nInput: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 290895\nOutput:', 'Input: 91011\nOutput: 09-10-11\n\nInput: 31108\nOutput: 03-11-08\n\nInput: 90508\nOutput: 09-05-08\n\nInput: 121099\nOutput: 12-10-99\n\nInput: 180413\nOutput:', 'Input: 13-03-1986\nOutput: 03/13/1986\n\nInput: 24/09/1984\nOutput: 09/24/1984\n\nInput: 2010-06-27\nOutput: 27/06/2010\n\nInput: 20.08.2010\nOutput: 08/20/2010\n\nInput: 17/06/1989\nOutput:', 'Input: 13-03-1986\nOutput: 03/13/1986\n\nInput: 24/09/1984\nOutput: 09/24/1984\n\nInput: 2010-06-27\nOutput: 27/06/2010\n\nInput: 20.08.2010\nOutp

In [ ]:
text_file = open("queries4.txt", "w")
for i in range(0, len(printed_queries)):
  text_file.write(repr(printed_queries[i]))
text_file.close()

## 3. Evaluate results

### GPT-3 DaVinci 1-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_1s_d.txt to results_dw_t0_1s_d.txt


In [ ]:
resdata = uploaded["results_dw_t0_1s_d.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets[i])
  for j in range(0, len(test_targets[i])):
    results[i][j] = resdata[i*len(test_targets[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.5008547008547009


In [ ]:
corr_num

293

In [ ]:
len(resdata)

585

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 40.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 40.0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 20.0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 20.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 20.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 80.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem getWeekDay-1 i

In [ ]:
sumAccuracy

58.60000000000029

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 40.95238095238095
GPT3 Accuracy for Domain emails: 88.88888888888889
GPT3 Accuracy for Domain freetext: 62.727272727272734
GPT3 Accuracy for Domain names: 55.38461538461539
GPT3 Accuracy for Domain phones: 63.33333333333333
GPT3 Accuracy for Domain times: 31.66666666666667
GPT3 Accuracy for Domain units: 20.0


### GPT-3 DaVinci 2-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_2s_d.txt to results_dw_t0_2s_d.txt


In [ ]:
resdata = uploaded["results_dw_t0_2s_d.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets2[i])
  for j in range(0, len(test_targets2[i])):
    results[i][j] = resdata[i*len(test_targets2[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.6282051282051282


In [ ]:
corr_num

294

In [ ]:
len(resdata)

468

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets2[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets2[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 75.0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 25.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 75.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 25.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 75.0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 75.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100.0

In [ ]:
sumAccuracy

73.5

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets2

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets2[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets2[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 57.14285714285714
GPT3 Accuracy for Domain emails: 94.44444444444444
GPT3 Accuracy for Domain freetext: 72.72727272727273
GPT3 Accuracy for Domain names: 63.46153846153846
GPT3 Accuracy for Domain phones: 73.61111111111111
GPT3 Accuracy for Domain times: 43.75
GPT3 Accuracy for Domain units: 50.0


### GPT-3 DaVinci 3-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_3s_d.txt to results_dw_t0_3s_d.txt


In [ ]:
resdata = uploaded["results_dw_t0_3s_d.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets3[i])
  for j in range(0, len(test_targets3[i])):
    results[i][j] = resdata[i*len(test_targets3[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.6324786324786325


In [ ]:
corr_num

222

In [ ]:
len(resdata)

351

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets3[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets3[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy 

In [ ]:
sumAccuracy

74.00000000000003

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets3

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets3[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets3[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 55.55555555555556
GPT3 Accuracy for Domain emails: 92.59259259259258
GPT3 Accuracy for Domain freetext: 74.24242424242424
GPT3 Accuracy for Domain names: 58.974358974358964
GPT3 Accuracy for Domain phones: 74.07407407407408
GPT3 Accuracy for Domain times: 44.44444444444444
GPT3 Accuracy for Domain units: 60.0


### GPT-3 DaVinci 4-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_4s_d.txt to results_dw_t0_4s_d.txt


In [ ]:
resdata = uploaded["results_dw_t0_4s_d.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets4[i])
  for j in range(0, len(test_targets4[i])):
    results[i][j] = resdata[i*len(test_targets4[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.6752136752136753


In [ ]:
corr_num

158

In [ ]:
len(resdata)

234

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets4[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets4[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 50.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100

In [ ]:
sumAccuracy

79.0

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets4

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets4[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets4[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 61.904761904761905
GPT3 Accuracy for Domain emails: 88.88888888888889
GPT3 Accuracy for Domain freetext: 70.45454545454545
GPT3 Accuracy for Domain names: 76.92307692307693
GPT3 Accuracy for Domain phones: 77.77777777777779
GPT3 Accuracy for Domain times: 54.166666666666664
GPT3 Accuracy for Domain units: 55.00000000000001


### GPT-3 Curie 1-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_1s_c.txt to results_dw_t0_1s_c.txt


In [ ]:
resdata = uploaded["results_dw_t0_1s_c.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets[i])
  for j in range(0, len(test_targets[i])):
    results[i][j] = resdata[i*len(test_targets[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.3811965811965812


In [ ]:
corr_num

223

In [ ]:
len(resdata)

585

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 40.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 40.0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 20.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 40.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 40.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in D

In [ ]:
sumAccuracy

44.600000000000094

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 34.28571428571429
GPT3 Accuracy for Domain emails: 86.66666666666667
GPT3 Accuracy for Domain freetext: 50.90909090909093
GPT3 Accuracy for Domain names: 44.61538461538462
GPT3 Accuracy for Domain phones: 35.555555555555564
GPT3 Accuracy for Domain times: 25.000000000000007
GPT3 Accuracy for Domain units: 2.0


### GPT-3 Curie 2-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_2s_c.txt to results_dw_t0_2s_c.txt


In [ ]:
resdata = uploaded["results_dw_t0_2s_c.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets2[i])
  for j in range(0, len(test_targets2[i])):
    results[i][j] = resdata[i*len(test_targets2[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.561965811965812


In [ ]:
corr_num

263

In [ ]:
len(resdata)

468

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets2[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets2[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 50.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 25.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 25.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 75.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 75.0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100.0
GPT

In [ ]:
sumAccuracy

65.75

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets2

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets2[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets2[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 51.19047619047619
GPT3 Accuracy for Domain emails: 83.33333333333334
GPT3 Accuracy for Domain freetext: 77.27272727272727
GPT3 Accuracy for Domain names: 67.3076923076923
GPT3 Accuracy for Domain phones: 48.61111111111111
GPT3 Accuracy for Domain times: 40.625
GPT3 Accuracy for Domain units: 32.5


### GPT-3 Curie 3-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_3s_c.txt to results_dw_t0_3s_c (1).txt


In [ ]:
resdata = uploaded["results_dw_t0_3s_c.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets3[i])
  for j in range(0, len(test_targets3[i])):
    results[i][j] = resdata[i*len(test_targets3[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.5925925925925926


In [ ]:
corr_num

208

In [ ]:
len(resdata)

351

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets3[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets3[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem getMonthName-2 in Do

In [ ]:
sumAccuracy

69.33333333333343

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets3

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets3[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets3[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 57.14285714285714
GPT3 Accuracy for Domain emails: 88.88888888888889
GPT3 Accuracy for Domain freetext: 69.69696969696969
GPT3 Accuracy for Domain names: 66.66666666666666
GPT3 Accuracy for Domain phones: 51.85185185185186
GPT3 Accuracy for Domain times: 48.6111111111111
GPT3 Accuracy for Domain units: 43.33333333333333


### GPT-3 Curie 4-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_4s_c.txt to results_dw_t0_4s_c.txt


In [ ]:
resdata = uploaded["results_dw_t0_4s_c.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets4[i])
  for j in range(0, len(test_targets4[i])):
    results[i][j] = resdata[i*len(test_targets4[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.6324786324786325


In [ ]:
corr_num

148

In [ ]:
len(resdata)

234

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets4[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets4[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100

In [ ]:
sumAccuracy

74.0

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets4

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets4[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets4[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 59.523809523809526
GPT3 Accuracy for Domain emails: 94.44444444444444
GPT3 Accuracy for Domain freetext: 72.72727272727273
GPT3 Accuracy for Domain names: 76.92307692307693
GPT3 Accuracy for Domain phones: 55.55555555555556
GPT3 Accuracy for Domain times: 54.166666666666664
GPT3 Accuracy for Domain units: 40.0


### GPT-3 Babbage 1-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_1s_b.txt to results_dw_t0_1s_b.txt


In [ ]:
resdata = uploaded["results_dw_t0_1s_b.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets[i])
  for j in range(0, len(test_targets[i])):
    results[i][j] = resdata[i*len(test_targets[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.3213675213675214


In [ ]:
corr_num

188

In [ ]:
len(resdata)

585

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 40.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 40.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 80.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 40.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 80.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates:

In [ ]:
sumAccuracy

37.599999999999994

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 26.666666666666668
GPT3 Accuracy for Domain emails: 57.77777777777779
GPT3 Accuracy for Domain freetext: 44.54545454545455
GPT3 Accuracy for Domain names: 26.153846153846157
GPT3 Accuracy for Domain phones: 27.77777777777779
GPT3 Accuracy for Domain times: 34.16666666666667
GPT3 Accuracy for Domain units: 4.0


### GPT-3 Babbage 2-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_2s_b.txt to results_dw_t0_2s_b.txt


In [ ]:
resdata = uploaded["results_dw_t0_2s_b.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets2[i])
  for j in range(0, len(test_targets2[i])):
    results[i][j] = resdata[i*len(test_targets2[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.49572649572649574


In [ ]:
corr_num

232

In [ ]:
len(resdata)

468

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets2[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets2[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 75.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 75.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 25.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 75.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100.0
GPT3 Ac

In [ ]:
sumAccuracy

58.0

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets2

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets2[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets2[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 40.476190476190474
GPT3 Accuracy for Domain emails: 91.66666666666666
GPT3 Accuracy for Domain freetext: 55.68181818181818
GPT3 Accuracy for Domain names: 50.0
GPT3 Accuracy for Domain phones: 51.388888888888886
GPT3 Accuracy for Domain times: 41.66666666666667
GPT3 Accuracy for Domain units: 32.5


### GPT-3 Babbage 3-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_3s_b.txt to results_dw_t0_3s_b.txt


In [ ]:
resdata = uploaded["results_dw_t0_3s_b.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets3[i])
  for j in range(0, len(test_targets3[i])):
    results[i][j] = resdata[i*len(test_targets3[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.5555555555555556


In [ ]:
corr_num

195

In [ ]:
len(resdata)

351

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets3[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets3[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 A

In [ ]:
sumAccuracy

65.00000000000016

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets3

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets3[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets3[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 42.857142857142854
GPT3 Accuracy for Domain emails: 88.88888888888889
GPT3 Accuracy for Domain freetext: 68.18181818181817
GPT3 Accuracy for Domain names: 64.1025641025641
GPT3 Accuracy for Domain phones: 61.111111111111114
GPT3 Accuracy for Domain times: 41.66666666666667
GPT3 Accuracy for Domain units: 36.666666666666664


### GPT-3 Babbage 4-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_4s_b.txt to results_dw_t0_4s_b.txt


In [ ]:
resdata = uploaded["results_dw_t0_4s_b.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets4[i])
  for j in range(0, len(test_targets4[i])):
    results[i][j] = resdata[i*len(test_targets4[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.5512820512820513


In [ ]:
corr_num

129

In [ ]:
len(resdata)

234

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets4[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets4[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 50.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100.0
G

In [ ]:
sumAccuracy

64.5

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets4

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets4[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets4[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 54.761904761904766
GPT3 Accuracy for Domain emails: 88.88888888888889
GPT3 Accuracy for Domain freetext: 61.36363636363637
GPT3 Accuracy for Domain names: 65.38461538461539
GPT3 Accuracy for Domain phones: 58.333333333333336
GPT3 Accuracy for Domain times: 37.5
GPT3 Accuracy for Domain units: 35.0


### GPT-3 Ada 1-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_1s_a.txt to results_dw_t0_1s_a.txt


In [ ]:
resdata = uploaded["results_dw_t0_1s_a.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets[i])
  for j in range(0, len(test_targets[i])):
    results[i][j] = resdata[i*len(test_targets[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.2358974358974359


In [ ]:
corr_num

138

In [ ]:
len(resdata)

585

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 60.00000000000001
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 80.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 20.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 20.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 20.0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 80.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 0
GPT3 Accuracy

In [ ]:
sumAccuracy

27.599999999999934

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 15.238095238095239
GPT3 Accuracy for Domain emails: 37.77777777777778
GPT3 Accuracy for Domain freetext: 39.0909090909091
GPT3 Accuracy for Domain names: 20.000000000000004
GPT3 Accuracy for Domain phones: 16.666666666666664
GPT3 Accuracy for Domain times: 27.500000000000007
GPT3 Accuracy for Domain units: 2.0


### GPT-3 Ada 2-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_2s_a.txt to results_dw_t0_2s_a.txt


In [ ]:
resdata = uploaded["results_dw_t0_2s_a.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets2[i])
  for j in range(0, len(test_targets2[i])):
    results[i][j] = resdata[i*len(test_targets2[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.43162393162393164


In [ ]:
corr_num

202

In [ ]:
len(resdata)

468

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets2

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets2[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets2[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets2[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 25.0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 75.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 75.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 25.0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 25.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100.0
GPT3 Ac

In [ ]:
sumAccuracy

50.5

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets2

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets2[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets2[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 36.904761904761905
GPT3 Accuracy for Domain emails: 83.33333333333334
GPT3 Accuracy for Domain freetext: 43.18181818181818
GPT3 Accuracy for Domain names: 53.84615384615385
GPT3 Accuracy for Domain phones: 37.5
GPT3 Accuracy for Domain times: 39.58333333333333
GPT3 Accuracy for Domain units: 25.0


### GPT-3 Ada 3-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_3s_a.txt to results_dw_t0_3s_a.txt


In [ ]:
resdata = uploaded["results_dw_t0_3s_a.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets3[i])
  for j in range(0, len(test_targets3[i])):
    results[i][j] = resdata[i*len(test_targets3[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.4757834757834758


In [ ]:
corr_num

167

In [ ]:
len(resdata)

351

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets3

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets3[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets3[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets3[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 33.33333333333333
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 66.66666666666666
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Prob

In [ ]:
sumAccuracy

55.666666666666785

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets3

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets3[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets3[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 39.682539682539684
GPT3 Accuracy for Domain emails: 81.48148148148148
GPT3 Accuracy for Domain freetext: 56.06060606060607
GPT3 Accuracy for Domain names: 41.02564102564102
GPT3 Accuracy for Domain phones: 44.44444444444444
GPT3 Accuracy for Domain times: 45.83333333333333
GPT3 Accuracy for Domain units: 33.33333333333333


### GPT-3 Ada 4-shot with Temp = 0// Prompt WITHOUT the task description

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving results_dw_t0_4s_a.txt to results_dw_t0_4s_a.txt


In [ ]:
resdata = uploaded["results_dw_t0_4s_a.txt"].decode("utf-8").split("\n")

In [ ]:
resdata = resdata[:-1]

In [ ]:
results = [[]]*len(final_problems)
for i in range(0, len(final_problems)):
  results[i] = ['']*len(test_targets4[i])
  for j in range(0, len(test_targets4[i])):
    results[i][j] = resdata[i*len(test_targets4[i]) + j]

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

corr_num = 0
 
#Calculate the accuracies
for i in range(0, len(final_problems)):
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        corr_num = corr_num + 1

print(corr_num / len(resdata))

0.5


In [ ]:
corr_num

117

In [ ]:
len(resdata)

234

In [ ]:
#Evaluation (T-0)
 
correctAnswers = test_targets4

sumAccuracy = 0
#Calculate the accuracies
for i in range(0, len(final_problems)):
  accuracy = 0
  for j in range(0, len(test_targets4[i])):
    if results[i][j] == correctAnswers[i][j]:
        accuracy = accuracy + (1/len(test_targets4[i]))
        sumAccuracy = sumAccuracy + (1/len(test_targets4[i]))
  print("GPT3 Accuracy for Problem " + final_problems[i][0] + " in Domain " + final_problems[i][1] + ": " + str(accuracy*100))

GPT3 Accuracy for Problem addPunctuation-1 in Domain dates: 50.0
GPT3 Accuracy for Problem addPunctuation-2 in Domain dates: 50.0
GPT3 Accuracy for Problem changeFormat-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changeFormat-2 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-3 in Domain dates: 0
GPT3 Accuracy for Problem changeFormat-4 in Domain dates: 0
GPT3 Accuracy for Problem changePunctuation-1 in Domain dates: 100.0
GPT3 Accuracy for Problem changePunctuation-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-1 in Domain dates: 0
GPT3 Accuracy for Problem getDay-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getDay-3 in Domain dates: 100.0
GPT3 Accuracy for Problem getDayOrdinal-1 in Domain dates: 0
GPT3 Accuracy for Problem getDayOrdinal-2 in Domain dates: 0
GPT3 Accuracy for Problem getMonthName-1 in Domain dates: 50.0
GPT3 Accuracy for Problem getMonthName-2 in Domain dates: 100.0
GPT3 Accuracy for Problem getWeekDay-1 in Domain dates: 100.0
GPT3 Ac

In [ ]:
sumAccuracy

58.5

In [ ]:
#Evaluation (T-0) by domain
 
correctAnswers = test_targets4

sum = 0
sumAccuracy = 0
#Calculate the accuracies
for d in range(0, len(domains)):
  for i in range(0, len(final_problems)):
    if final_problems[i][1] == domains[d]:
      sum = sum + 1
      accuracy = 0
      for j in range(0, len(test_targets4[i])):
        if results[i][j] == correctAnswers[i][j]:
            accuracy = accuracy + (1/len(test_targets4[i]))
      sumAccuracy = sumAccuracy + accuracy
  print("GPT3 Accuracy for Domain " + domains[d] + ": " + str((sumAccuracy/sum)*100))
  sum = 0
  sumAccuracy = 0

GPT3 Accuracy for Domain dates: 50.0
GPT3 Accuracy for Domain emails: 83.33333333333334
GPT3 Accuracy for Domain freetext: 52.27272727272727
GPT3 Accuracy for Domain names: 53.84615384615385
GPT3 Accuracy for Domain phones: 52.77777777777778
GPT3 Accuracy for Domain times: 37.5
GPT3 Accuracy for Domain units: 35.0
